In [5]:
#!/usr/bin/env python3
"""
smc_signals_v3.py
- Explicit alignment of every Series to df.index
- Defensive helpers for boolean logic and np.where usage
- Optional DEBUG prints to trace any misalignment
"""

from __future__ import annotations
import math
import sys
from dataclasses import dataclass
from typing import List, Dict, Tuple
import numpy as np
import pandas as pd

# =========================
# CONFIG
# =========================
TICKERS         = ['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'APLLTD.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'ARE&M.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AIIL.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BLS.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BERGEPAINT.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BSOFT.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'FIRSTCRY.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'MAPMYINDIA.NS', 'CCL.NS', 'CESC.NS', 'CGPOWER.NS', 'CRISIL.NS', 'CAMPUS.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CAPLIPOINT.NS', 'CGCL.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'CUB.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CAMS.NS', 'CONCORDBIO.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CROMPTON.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DCMSHRIRAM.NS', 'DLF.NS', 'DOMS.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'LALPATHLAB.NS', 'DRREDDY.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EICHERMOT.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'NYKAA.NS', 'FEDERALBNK.NS', 'FACT.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FSL.NS', 'FIVESTAR.NS', 'FORTIS.NS', 'GAIL.NS', 'GVT&D.NS', 'GMRAIRPORT.NS', 'GRSE.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'MEDANTA.NS', 'GODIGIT.NS', 'GPIL.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GESHIP.NS', 'FLUOROCHEM.NS', 'GUJGASLTD.NS', 'GMDCLTD.NS', 'GNFC.NS', 'GPPL.NS', 'GSPL.NS', 'HEG.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HEROMOTOCO.NS', 'HSCL.NS', 'HINDALCO.NS', 'HAL.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'POWERINDIA.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IFCI.NS', 'IIFL.NS', 'INOXINDIA.NS', 'IRB.NS', 'IRCON.NS', 'ITC.NS', 'ITI.NS', 'INDGN.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'IEX.NS', 'INDHOTEL.NS', 'IOC.NS', 'IOB.NS', 'IRCTC.NS', 'IRFC.NS', 'IREDA.NS', 'IGL.NS', 'INDUSTOWER.NS', 'INDUSINDBK.NS', 'NAUKRI.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'INDIGO.NS', 'IGIL.NS', 'IKS.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'JKCEMENT.NS', 'JBMA.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JPPOWER.NS', 'J&KBANK.NS', 'JINDALSAW.NS', 'JSL.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KPRMILL.NS', 'KEI.NS', 'KNRCON.NS', 'KPITTECH.NS', 'KAJARIACER.NS', 'KPIL.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KFINTECH.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KIMS.NS', 'LTF.NS', 'LTTS.NS', 'LICHSGFIN.NS', 'LTFOODS.NS', 'LTIM.NS', 'LT.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LUPIN.NS', 'MMTC.NS', 'MRF.NS', 'MGL.NS', 'MAHSEAMLES.NS', 'M&MFIN.NS', 'M&M.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MANKIND.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'METROPOLIS.NS', 'MINDACORP.NS', 'MSUMI.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NCC.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPCGREEN.NS', 'NTPC.NS', 'NH.NS', 'NATIONALUM.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NAM-INDIA.NS', 'NIVABUPA.NS', 'NUVAMA.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'PAYTM.NS', 'OFSS.NS', 'POLICYBZR.NS', 'PCBL.NS', 'PGEL.NS', 'PIIND.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'PTCIL.NS', 'PVRINOX.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFIZER.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PEL.NS', 'PPLPHARMA.NS', 'POLYMED.NS', 'POLYCAB.NS', 'POONAWALLA.NS', 'PFC.NS', 'POWERGRID.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PNB.NS', 'RRKABEL.NS', 'RBLBANK.NS', 'RECLTD.NS', 'RHIM.NS', 'RITES.NS', 'RADICO.NS', 'RVNL.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RKFORGE.NS', 'RCF.NS', 'RTNINDIA.NS', 'RAYMONDLSL.NS', 'RAYMOND.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RPOWER.NS', 'ROUTE.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SRF.NS', 'SAGILITY.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'MOTHERSON.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'RENUKA.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'STARHEALTH.NS', 'SBIN.NS', 'SAIL.NS', 'SWSOLAR.NS', 'SUMICHEM.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANENERGY.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TBOTEK.NS', 'TVSMOTOR.NS', 'TANLA.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TTML.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'NIACL.NS', 'RAMCOCEM.NS', 'THERMAX.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TARIL.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRIVENI.NS', 'TRITURBINE.NS', 'TIINDIA.NS', 'UCOBANK.NS', 'UNOMINDA.NS', 'UPL.NS', 'UTIAMC.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UBL.NS', 'UNITDSPR.NS', 'USHAMART.NS', 'VGUARD.NS', 'DBREALTY.NS', 'VTL.NS', 'VBL.NS', 'MANYAVAR.NS', 'VEDL.NS', 'VIJAYA.NS', 'VMM.NS', 'IDEA.NS', 'VOLTAS.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZFCVINDIA.NS', 'ZEEL.NS', 'ZENTEC.NS', 'ZENSARTECH.NS', 'ZYDUSLIFE.NS', 'ECLERX.NS']

START_DATE      = "2017-01-01"
END_DATE        = None
INTERVAL        = "1d"

SWING_LEN       = 20
INTERNAL_LEN    = 5
ATR_PERIOD      = 14
RR_TARGET       = 2.0
STOP_BUFFER_PCT = 0.001
MAX_HOLD_DAYS   = 30
RUN_BACKTEST    = True
DEBUG           = True

# =========================
# Helpers
# =========================

def A(x: pd.Series, idx: pd.Index, name=None):
    """Align series to idx and return copy."""
    if not isinstance(x, pd.Series):
        x = pd.Series(x, index=idx, name=name)
    if name and x.name != name:
        x = x.rename(name)
    return x.reindex(idx)

def atr(df: pd.DataFrame, period: int = 14) -> pd.Series:
    high, low, close = df["High"], df["Low"], df["Close"]
    prev_close = close.shift(1)
    tr = pd.concat([(high - low).abs(),
                    (high - prev_close).abs(),
                    (low - prev_close).abs()], axis=1).max(axis=1)
    return tr.rolling(period, min_periods=1).mean().reindex(df.index)

def pivot_high(series: pd.Series, left: int, right: int) -> pd.Series:
    s = series.copy()
    ph_left  = s.rolling(left+1, min_periods=1).max()
    ph_right = s[::-1].rolling(right+1, min_periods=1).max()[::-1]
    return (s.eq(ph_left) & s.eq(ph_right)).reindex(series.index)

def pivot_low(series: pd.Series, left: int, right: int) -> pd.Series:
    s = series.copy()
    pl_left  = s.rolling(left+1, min_periods=1).min()
    pl_right = s[::-1].rolling(right+1, min_periods=1).min()[::-1]
    return (s.eq(pl_left) & s.eq(pl_right)).reindex(series.index)

def detect_swings(df: pd.DataFrame, swing_len: int) -> pd.DataFrame:
    hi, lo = df["High"], df["Low"]
    ph = pivot_high(hi, swing_len, swing_len)
    pl = pivot_low(lo, swing_len, swing_len)
    out = pd.DataFrame(index=df.index)
    out["swing_high"] = hi.where(ph, np.nan)
    out["swing_low"]  = lo.where(pl, np.nan)
    out["last_swing_high"] = out["swing_high"].ffill()
    out["last_swing_low"]  = out["swing_low"].ffill()
    return out

def detect_internal(df: pd.DataFrame, internal_len: int) -> pd.DataFrame:
    hi, lo = df["High"], df["Low"]
    ph = pivot_high(hi, internal_len, internal_len)
    pl = pivot_low(lo, internal_len, internal_len)
    out = pd.DataFrame(index=df.index)
    out["int_high"] = hi.where(ph, np.nan)
    out["int_low"]  = lo.where(pl, np.nan)
    out["last_int_high"] = out["int_high"].ffill()
    out["last_int_low"]  = out["int_low"].ffill()
    return out

def detect_fvg(df: pd.DataFrame) -> pd.DataFrame:
    hi, lo = df["High"], df["Low"]
    bull = (lo > hi.shift(2)).fillna(False)
    bear = (hi < lo.shift(2)).fillna(False)
    return pd.DataFrame({"fvg_bull": bull, "fvg_bear": bear}, index=df.index)

def structure_bos_choch(df: pd.DataFrame, swings: pd.DataFrame) -> pd.DataFrame:
    idx = df.index
    c = df["Close"]
    last_high = swings["last_swing_high"]
    last_low  = swings["last_swing_low"]
    bos_up = A(c, idx) > A(last_high.shift(1), idx)
    bos_dn = A(c, idx) < A(last_low.shift(1), idx)
    bos_up = bos_up.fillna(False)
    bos_dn = bos_dn.fillna(False)

    bias = pd.Series(np.nan, index=idx)
    bias = bias.mask(bos_up, 1).mask(bos_dn, -1).ffill().fillna(0).astype(int)
    choch_up = (bos_up & (bias.shift(1) == -1)).fillna(False)
    choch_dn = (bos_dn & (bias.shift(1) ==  1)).fillna(False)

    out = pd.DataFrame(index=idx)
    out["bos_up"] = bos_up
    out["bos_dn"] = bos_dn
    out["bias"] = bias
    out["choch_up"] = choch_up
    out["choch_dn"] = choch_dn
    return out

def dealing_range(swings: pd.DataFrame) -> pd.DataFrame:
    top = swings["last_swing_high"]
    bot = swings["last_swing_low"]
    mid = (top + bot) / 2.0
    return pd.DataFrame({"dr_top": top, "dr_bot": bot, "dr_mid": mid}, index=swings.index)

@dataclass
class Trade:
    side: str
    entry_time: pd.Timestamp
    entry: float
    stop: float
    tp: float
    exit_time: pd.Timestamp | None = None
    exit: float | None = None
    outcome: str | None = None
    pnl: float | None = None

def generate_signals(df: pd.DataFrame) -> pd.DataFrame:
    idx = df.index

    atr_s  = A(atr(df, ATR_PERIOD), idx, "atr")
    swings = detect_swings(df, SWING_LEN)
    intern = detect_internal(df, INTERNAL_LEN)
    struct = structure_bos_choch(df, swings)
    dr     = dealing_range(swings)
    fvg    = detect_fvg(df)

    roll_low  = A(intern["last_int_low"].rolling(INTERNAL_LEN, min_periods=1).min(), idx)
    roll_high = A(intern["last_int_high"].rolling(INTERNAL_LEN, min_periods=1).max(), idx)

    sweep_long  = (A(df["Low"], idx)  < roll_low.shift(1)).fillna(False)
    sweep_short = (A(df["High"], idx) > roll_high.shift(1)).fillna(False)

    recent_bos_up = A(struct["bos_up"].rolling(10, min_periods=1).max().astype(bool), idx).fillna(False)
    recent_bos_dn = A(struct["bos_dn"].rolling(10, min_periods=1).max().astype(bool), idx).fillna(False)

    conf_long  = ((A(df["Close"], idx) > A(intern["last_int_high"].shift(1), idx)) | A(fvg["fvg_bull"], idx)).fillna(False)
    conf_short = ((A(df["Close"], idx) < A(intern["last_int_low"].shift(1), idx))  | A(fvg["fvg_bear"], idx)).fillna(False)

    in_discount = (A(df["Close"], idx) < A(dr["dr_mid"], idx)).fillna(False)
    in_premium  = (A(df["Close"], idx) > A(dr["dr_mid"], idx)).fillna(False)

    long_entry  = (recent_bos_up & in_discount & sweep_long & conf_long).astype(bool)
    short_entry = (recent_bos_dn & in_premium  & sweep_short & conf_short).astype(bool)

    entry_px   = A(df["Open"].shift(-1), idx).astype(float)
    swing_low  = A(swings["last_swing_low"], idx).astype(float)
    swing_high = A(swings["last_swing_high"], idx).astype(float)

    long_stop  = swing_low * (1.0 - STOP_BUFFER_PCT)
    short_stop = swing_high * (1.0 + STOP_BUFFER_PCT)

    long_risk  = entry_px - long_stop
    short_risk = short_stop - entry_px

    sig = pd.DataFrame(index=idx)
    sig["long_entry"]   = long_entry
    sig["short_entry"]  = short_entry
    sig["bias"]         = A(struct["bias"], idx).astype(int)
    sig["atr"]          = atr_s
    sig["dr_top"]       = A(dr["dr_top"], idx)
    sig["dr_bot"]       = A(dr["dr_bot"], idx)
    sig["dr_mid"]       = A(dr["dr_mid"], idx)
    sig["swing_low"]    = swing_low
    sig["swing_high"]   = swing_high
    sig["long_entry_px"]= np.where(long_entry.values, entry_px.values, np.nan)
    sig["long_stop"]    = np.where(long_entry.values, long_stop.values, np.nan)
    sig["long_tp"]      = np.where(long_entry.values, (entry_px + RR_TARGET * long_risk).values, np.nan)
    sig["short_entry_px"]=np.where(short_entry.values, entry_px.values, np.nan)
    sig["short_stop"]   = np.where(short_entry.values, short_stop.values, np.nan)
    sig["short_tp"]     = np.where(short_entry.values, (entry_px - RR_TARGET * short_risk).values, np.nan)

    # convert numpy arrays back to Series aligned to idx
    for col in ["long_entry_px","long_stop","long_tp","short_entry_px","short_stop","short_tp"]:
        sig[col] = pd.Series(sig[col], index=idx)

    if DEBUG:
        # sanity checks
        for name in ["long_entry","short_entry","bias","atr","dr_mid"]:
            s = sig[name]
            if not s.index.equals(idx):
                print(f"[DEBUG] {name} index misaligned", file=sys.stderr)

    return sig

def run_backtest(df: pd.DataFrame, sig: pd.DataFrame) -> List[Trade]:
    trades: List[Trade] = []
    o, h, l, c = df["Open"], df["High"], df["Low"], df["Close"]
    n = len(df)
    for i in range(n-1):
        if bool(sig["long_entry"].iloc[i]):
            entry_t = df.index[i+1]
            entry   = float(o.iloc[i+1])
            stop    = float(sig["long_stop"].iloc[i])
            tp      = float(sig["long_tp"].iloc[i])
            if not (math.isnan(entry) or math.isnan(stop) or math.isnan(tp)):
                trades.append(Trade("long", entry_t, entry, stop, tp))
        if bool(sig["short_entry"].iloc[i]):
            entry_t = df.index[i+1]
            entry   = float(o.iloc[i+1])
            stop    = float(sig["short_stop"].iloc[i])
            tp      = float(sig["short_tp"].iloc[i])
            if not (math.isnan(entry) or math.isnan(stop) or math.isnan(tp)):
                trades.append(Trade("short", entry_t, entry, stop, tp))

        bar_t = df.index[i+1]
        hi, lo = float(h.iloc[i+1]), float(l.iloc[i+1])

        for t in trades:
            if t.exit is not None or bar_t < t.entry_time:
                continue
            # time exit
            if (bar_t - t.entry_time).days >= MAX_HOLD_DAYS:
                t.exit_time = bar_t; t.exit = float(c.iloc[i+1]); t.outcome = "time"
                t.pnl = (t.exit - t.entry) if t.side == "long" else (t.entry - t.exit)
                continue
            if t.side == "long":
                hit_sl = lo <= t.stop; hit_tp = hi >= t.tp
                if hit_sl or hit_tp:
                    t.exit_time = bar_t
                    if hit_sl and hit_tp: t.exit, t.outcome = t.stop, "sl"
                    elif hit_tp:          t.exit, t.outcome = t.tp, "tp"
                    else:                 t.exit, t.outcome = t.stop, "sl"
                    t.pnl = t.exit - t.entry
            else:
                hit_sl = hi >= t.stop; hit_tp = lo <= t.tp
                if hit_sl or hit_tp:
                    t.exit_time = bar_t
                    if hit_sl and hit_tp: t.exit, t.outcome = t.stop, "sl"
                    elif hit_tp:          t.exit, t.outcome = t.tp, "tp"
                    else:                 t.exit, t.outcome = t.stop, "sl"
                    t.pnl = t.entry - t.exit
    return trades

def trades_summary(trades: List[Trade]) -> Dict[str, float]:
    if not trades: return {"trades": 0, "win_rate": 0.0, "avg_pnl": 0.0, "total_pnl": 0.0}
    wins  = sum(1 for t in trades if t.outcome == "tp")
    total = len(trades)
    avg   = float(np.mean([t.pnl for t in trades]))
    tot   = float(np.sum([t.pnl for t in trades]))
    return {"trades": total, "win_rate": wins/total, "avg_pnl": avg, "total_pnl": tot}

def fetch_yahoo(ticker: str, start: str, end: str | None, interval: str = "1d") -> pd.DataFrame:
    try:
        import yfinance as yf
    except Exception as e:
        raise RuntimeError("Please install yfinance: pip install yfinance") from e
    df = yf.download(ticker, start=start, end=end, interval=interval, auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty:
        raise ValueError(f"No data for {ticker}")
    # ensure a single-level DatetimeIndex and standard columns
    df = df.dropna()
    df = df[["Open","High","Low","Close","Volume"]].copy()
    df.index = pd.to_datetime(df.index)
    return df

def process_ticker(ticker: str):
    df = fetch_yahoo(ticker, START_DATE, END_DATE, INTERVAL)
    sig = generate_signals(df)
    trades = run_backtest(df, sig) if RUN_BACKTEST else []
    return df, sig, trades

def main():
    all_rows, bt_rows = [], []
    for t in TICKERS:
        try:
            df, sig, trades = process_ticker(t)
            out = pd.DataFrame({
                "ticker": t,
                "date": sig.index,
                "long_entry": sig["long_entry"].values,
                "short_entry": sig["short_entry"].values,
                "long_entry_px": sig["long_entry_px"].values,
                "long_stop": sig["long_stop"].values,
                "long_tp": sig["long_tp"].values,
                "short_entry_px": sig["short_entry_px"].values,
                "short_stop": sig["short_stop"].values,
                "short_tp": sig["short_tp"].values,
                "bias": sig["bias"].values,
            })
            all_rows.append(out)
            if RUN_BACKTEST:
                bt_rows.append({"ticker": t, **trades_summary(trades)})
        except Exception as e:
            print(f"[WARN] {t}: {e}", file=sys.stderr)

    if all_rows:
        all_df = pd.concat(all_rows, ignore_index=True)
        all_df.to_csv("smc_signals_output.csv", index=False)
        print("Saved signals to smc_signals_output.csv (rows:", len(all_df), ")")
    if RUN_BACKTEST and bt_rows:
        bt_df = pd.DataFrame(bt_rows)
        bt_df.to_csv("smc_backtest_summary.csv", index=False)
        print("Saved backtest summary to smc_backtest_summary.csv"); print(bt_df)

if __name__ == "__main__":
    main()


Saved signals to smc_signals_output.csv (rows: 903037 )
Saved backtest summary to smc_backtest_summary.csv
            ticker  trades  win_rate  avg_pnl  total_pnl
0        360ONE.NS       0       0.0      0.0        0.0
1       3MINDIA.NS       0       0.0      0.0        0.0
2           ABB.NS       0       0.0      0.0        0.0
3           ACC.NS       0       0.0      0.0        0.0
4     ACMESOLAR.NS       0       0.0      0.0        0.0
..             ...     ...       ...      ...        ...
495        ZEEL.NS       0       0.0      0.0        0.0
496      ZENTEC.NS       0       0.0      0.0        0.0
497  ZENSARTECH.NS       0       0.0      0.0        0.0
498   ZYDUSLIFE.NS       0       0.0      0.0        0.0
499      ECLERX.NS       0       0.0      0.0        0.0

[500 rows x 5 columns]


In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
SMC signal generator for multiple NSE stocks with smartmoneyconcepts integration.

- Uses smartmoneyconcepts (if available) for: swings, BOS/CHoCH, FVG, OB
- Falls back to built-in detectors if the package isn't importable (common on Python 3.12).
- Outputs:
    * smc_signals.csv  (all tickers)
    * ./plots/<TICKER>_smc_signals.png

Docs / API reference (functions used):
- from smartmoneyconcepts import smc
  - smc.swing_highs_lows(...)
  - smc.bos_choch(...)
  - smc.fvg(...)
  - smc.ob(...)
"""

from __future__ import annotations
import os
import math
import logging
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# =========================
# CONFIG (Edit these)
# =========================
TICKERS: List[str] = ['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'APLLTD.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'ARE&M.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AIIL.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BLS.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BERGEPAINT.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BSOFT.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'FIRSTCRY.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'MAPMYINDIA.NS', 'CCL.NS', 'CESC.NS', 'CGPOWER.NS', 'CRISIL.NS', 'CAMPUS.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CAPLIPOINT.NS', 'CGCL.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'CUB.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CAMS.NS', 'CONCORDBIO.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CROMPTON.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DCMSHRIRAM.NS', 'DLF.NS', 'DOMS.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'LALPATHLAB.NS', 'DRREDDY.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EICHERMOT.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'NYKAA.NS', 'FEDERALBNK.NS', 'FACT.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FSL.NS', 'FIVESTAR.NS', 'FORTIS.NS', 'GAIL.NS', 'GVT&D.NS', 'GMRAIRPORT.NS', 'GRSE.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'MEDANTA.NS', 'GODIGIT.NS', 'GPIL.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GESHIP.NS', 'FLUOROCHEM.NS', 'GUJGASLTD.NS', 'GMDCLTD.NS', 'GNFC.NS', 'GPPL.NS', 'GSPL.NS', 'HEG.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HEROMOTOCO.NS', 'HSCL.NS', 'HINDALCO.NS', 'HAL.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'POWERINDIA.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IFCI.NS', 'IIFL.NS', 'INOXINDIA.NS', 'IRB.NS', 'IRCON.NS', 'ITC.NS', 'ITI.NS', 'INDGN.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'IEX.NS', 'INDHOTEL.NS', 'IOC.NS', 'IOB.NS', 'IRCTC.NS', 'IRFC.NS', 'IREDA.NS', 'IGL.NS', 'INDUSTOWER.NS', 'INDUSINDBK.NS', 'NAUKRI.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'INDIGO.NS', 'IGIL.NS', 'IKS.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'JKCEMENT.NS', 'JBMA.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JPPOWER.NS', 'J&KBANK.NS', 'JINDALSAW.NS', 'JSL.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KPRMILL.NS', 'KEI.NS', 'KNRCON.NS', 'KPITTECH.NS', 'KAJARIACER.NS', 'KPIL.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KFINTECH.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KIMS.NS', 'LTF.NS', 'LTTS.NS', 'LICHSGFIN.NS', 'LTFOODS.NS', 'LTIM.NS', 'LT.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LUPIN.NS', 'MMTC.NS', 'MRF.NS', 'MGL.NS', 'MAHSEAMLES.NS', 'M&MFIN.NS', 'M&M.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MANKIND.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'METROPOLIS.NS', 'MINDACORP.NS', 'MSUMI.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NCC.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPCGREEN.NS', 'NTPC.NS', 'NH.NS', 'NATIONALUM.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NAM-INDIA.NS', 'NIVABUPA.NS', 'NUVAMA.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'PAYTM.NS', 'OFSS.NS', 'POLICYBZR.NS', 'PCBL.NS', 'PGEL.NS', 'PIIND.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'PTCIL.NS', 'PVRINOX.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFIZER.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PEL.NS', 'PPLPHARMA.NS', 'POLYMED.NS', 'POLYCAB.NS', 'POONAWALLA.NS', 'PFC.NS', 'POWERGRID.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PNB.NS', 'RRKABEL.NS', 'RBLBANK.NS', 'RECLTD.NS', 'RHIM.NS', 'RITES.NS', 'RADICO.NS', 'RVNL.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RKFORGE.NS', 'RCF.NS', 'RTNINDIA.NS', 'RAYMONDLSL.NS', 'RAYMOND.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RPOWER.NS', 'ROUTE.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SRF.NS', 'SAGILITY.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'MOTHERSON.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'RENUKA.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'STARHEALTH.NS', 'SBIN.NS', 'SAIL.NS', 'SWSOLAR.NS', 'SUMICHEM.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANENERGY.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TBOTEK.NS', 'TVSMOTOR.NS', 'TANLA.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TTML.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'NIACL.NS', 'RAMCOCEM.NS', 'THERMAX.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TARIL.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRIVENI.NS', 'TRITURBINE.NS', 'TIINDIA.NS', 'UCOBANK.NS', 'UNOMINDA.NS', 'UPL.NS', 'UTIAMC.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UBL.NS', 'UNITDSPR.NS', 'USHAMART.NS', 'VGUARD.NS', 'DBREALTY.NS', 'VTL.NS', 'VBL.NS', 'MANYAVAR.NS', 'VEDL.NS', 'VIJAYA.NS', 'VMM.NS', 'IDEA.NS', 'VOLTAS.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZFCVINDIA.NS', 'ZEEL.NS', 'ZENTEC.NS', 'ZENSARTECH.NS', 'ZYDUSLIFE.NS', 'ECLERX.NS']

START_DATE: str     = "2018-01-01"
END_DATE: Optional[str] = None      # None = till today
INTERVAL: str       = "1d"          # e.g., "1d", "1h", "30m", "15m"
TIMEZONE: str       = "Asia/Kolkata"

# Structure / zone params
SWING_LEN_LIB: int  = 10            # for smartmoneyconcepts.swing_highs_lows
PIVOT_LEFT_FB: int  = 2             # fallback fractal
PIVOT_RIGHT_FB: int = 2
BOS_LOOKBACK: int   = 200           # how far back to search for the most recent BOS
VALID_ZONE_BARS: int = 200          # FVG/OB "freshness" (bars)

# Entry behavior
ENTRY_MODE: str     = "retest_or_follow"
#   "retest_only"      -> require touch of bullish/bearish FVG/OB
#   "retest_or_follow" -> also allow quick follow-through after BOS/CHoCH

# Risk/Target
ATR_WINDOW: int     = 14
ATR_MULT_SL: float  = 0.25          # ATR cushion added/subtracted to SL
R_MULT_TP: float    = 1.5           # take-profit multiple of risk
MAX_OPEN_TRADES_PER_TICKER: int = 1

# Plot / Output
PLOT_DIR: str       = "plots"
PLOT_MAX_BARS: int  = 500
OUT_CSV: str        = "smc_signals.csv"

# Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
log = logging.getLogger("SMC")

# Try to import smartmoneyconcepts
USE_SMC_LIB = False
try:
    from smartmoneyconcepts import smc  # pip install smartmoneyconcepts
    USE_SMC_LIB = True
    log.info("smartmoneyconcepts detected. Using library indicators where possible.")
except Exception as e:
    log.warning("smartmoneyconcepts not available (%s). Using fallback detectors.", e)


# =========================
# Data structures
# =========================
@dataclass
class Zone:
    kind: str                  # 'FVG_BULL','FVG_BEAR','OB_BULL','OB_BEAR'
    created_at: pd.Timestamp
    lower: float
    upper: float
    valid_until: pd.Timestamp

    def overlaps(self, low: float, high: float) -> bool:
        return not (high < self.lower or low > self.upper)

@dataclass
class Position:
    side: str                  # "long" or "short"
    entry_time: pd.Timestamp
    entry_price: float
    stop_loss: float
    take_profit: float
    reason: str
    open: bool = True
    risk_per_share: float = 0.0

    def check_exit(
        self, t: pd.Timestamp, o: float, h: float, l: float, c: float, structure_flip: bool
    ) -> Optional[Tuple[str, pd.Timestamp, float, float]]:
        """Return (signal, time, price, r_multiple) if an exit occurs."""
        if not self.open:
            return None

        if self.side == "long":
            # SL first
            if l <= self.stop_loss:
                self.open = False
                r = (self.stop_loss - self.entry_price) / self.risk_per_share if self.risk_per_share else np.nan
                return ("LONG_EXIT", t, self.stop_loss, r)
            # TP second
            if h >= self.take_profit:
                self.open = False
                r = (self.take_profit - self.entry_price) / self.risk_per_share if self.risk_per_share else np.nan
                return ("LONG_EXIT", t, self.take_profit, r)
            # Structure flip (bearish) at close
            if structure_flip:
                self.open = False
                r = (c - self.entry_price) / self.risk_per_share if self.risk_per_share else np.nan
                return ("LONG_EXIT", t, c, r)
        else:
            # short
            if h >= self.stop_loss:
                self.open = False
                r = (self.entry_price - self.stop_loss) / self.risk_per_share if self.risk_per_share else np.nan
                return ("SHORT_EXIT", t, self.stop_loss, r)
            if l <= self.take_profit:
                self.open = False
                r = (self.entry_price - self.take_profit) / self.risk_per_share if self.risk_per_share else np.nan
                return ("SHORT_EXIT", t, self.take_profit, r)
            if structure_flip:
                self.open = False
                r = (self.entry_price - c) / self.risk_per_share if self.risk_per_share else np.nan
                return ("SHORT_EXIT", t, c, r)

        return None


# =========================
# Utilities
# =========================
def to_lower_ohlcv(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure columns are lower case for smartmoneyconcepts: open/high/low/close/volume."""
    mapper = {
        "Open": "open", "High": "high", "Low": "low", "Close": "close",
        "Adj Close": "adj close", "Volume": "volume"
    }
    out = df.rename(columns=mapper)
    for col in ["open","high","low","close"]:
        if col not in out.columns:
            raise ValueError("Missing OHLC column: " + col)
    if "volume" not in out.columns:
        out["volume"] = np.nan
    return out

def compute_atr(df: pd.DataFrame, window: int = 14) -> pd.Series:
    high, low, close = df["high"], df["low"], df["close"]
    prev_close = close.shift(1)
    tr = np.maximum(high - low, np.maximum((high - prev_close).abs(), (low - prev_close).abs()))
    return tr.rolling(window=window, min_periods=1).mean()

def fetch_history(ticker: str, start: str, end: Optional[str], interval: str) -> pd.DataFrame:
    try:
        df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, auto_adjust=False, multi_level_index=False)
    except Exception as e:
        log.error("Download failed for %s: %s", ticker, e)
        return pd.DataFrame()
    if df is None or df.empty:
        log.warning("No data for %s", ticker)
        return pd.DataFrame()

    # Normalize timezone
    if df.index.tz is None:
        df = df.tz_localize(TIMEZONE)
    else:
        df = df.tz_convert(TIMEZONE)

    # Lower-case columns for lib
    df = df.rename(columns=str.title)
    df = df.dropna(subset=["Open", "High", "Low", "Close"])
    df_lc = to_lower_ohlcv(df)
    return df_lc


# =========================
# Fallback detectors (when smartmoneyconcepts is unavailable)
# =========================
def fb_find_swings(df: pd.DataFrame, left: int = 2, right: int = 2) -> pd.DataFrame:
    n = len(df)
    sh = np.zeros(n, dtype=bool)
    sl = np.zeros(n, dtype=bool)
    H, L = df["high"].values, df["low"].values
    for i in range(left, n - right):
        if H[i] == H[i-left:i+right+1].max() and (H[i] > H[i-left:i] .max() or H[i] > H[i+1:i+right+1].max()):
            sh[i] = True
        if L[i] == L[i-left:i+right+1].min() and (L[i] < L[i-left:i] .min() or L[i] < L[i+1:i+right+1].min()):
            sl[i] = True
    out = pd.DataFrame(index=df.index)
    out["HighLow"] = 0
    out.loc[out.index[sh], "HighLow"] = 1
    out.loc[out.index[sl], "HighLow"] = -1
    out["Level"] = np.where(out["HighLow"]==1, df["high"], np.where(out["HighLow"]==-1, df["low"], np.nan))
    return out

def fb_bos_choch(df: pd.DataFrame, swings: pd.DataFrame, close_break: bool = True) -> pd.DataFrame:
    out = pd.DataFrame(index=df.index, columns=["BOS","CHOCH","Level","BrokenIndex"])
    out[["BOS","CHOCH"]] = 0
    out["Level"] = np.nan
    out["BrokenIndex"] = np.nan

    last_trend = 0  # 1 bull, -1 bear, 0 neutral
    last_high = np.nan
    last_low = np.nan

    for i, t in enumerate(df.index):
        if swings["HighLow"].iloc[i] == 1:
            last_high = swings["Level"].iloc[i]
        if swings["HighLow"].iloc[i] == -1:
            last_low = swings["Level"].iloc[i]

        price = df["close"].iloc[i] if close_break else df["high"].iloc[i]
        if pd.notna(last_high) and price >= last_high:
            out.loc[t, "BOS"] = 1
            out.loc[t, "Level"] = last_high
            out.loc[t, "BrokenIndex"] = i
            if last_trend != 1:
                out.loc[t, "CHOCH"] = 1
            last_trend = 1

        price = df["close"].iloc[i] if close_break else df["low"].iloc[i]
        if pd.notna(last_low) and price <= last_low:
            out.loc[t, "BOS"] = -1
            out.loc[t, "Level"] = last_low
            out.loc[t, "BrokenIndex"] = i
            if last_trend != -1:
                out.loc[t, "CHOCH"] = -1
            last_trend = -1
    return out

def fb_fvg(df: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame(index=df.index, columns=["FVG","Top","Bottom","MitigatedIndex"])
    out[["FVG"]] = 0
    out[["Top","Bottom","MitigatedIndex"]] = np.nan
    for i in range(1, len(df)-1):
        prev_h, next_l = df["high"].iloc[i-1], df["low"].iloc[i+1]
        prev_l, next_h = df["low"].iloc[i-1], df["high"].iloc[i+1]
        if next_l > prev_h:   # bullish FVG
            out.iloc[i, out.columns.get_loc("FVG")] = 1
            out.iloc[i, out.columns.get_loc("Top")] = next_l
            out.iloc[i, out.columns.get_loc("Bottom")] = prev_h
        if next_h < prev_l:   # bearish FVG
            out.iloc[i, out.columns.get_loc("FVG")] = -1
            out.iloc[i, out.columns.get_loc("Top")] = prev_l
            out.iloc[i, out.columns.get_loc("Bottom")] = next_h
    return out

def fb_ob(df: pd.DataFrame, swings: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame(index=df.index, columns=["OB","Top","Bottom","OBVolume","Percentage"])
    out[["OB"]] = 0
    out[["Top","Bottom","OBVolume","Percentage"]] = np.nan

    # Very simple OB: body of last opposite-color candle before BOS-like wide candle
    for i in range(2, len(df)):
        # bullish body > avg and prior candle bearish => bull OB at prior candle body
        body = abs(df["close"].iloc[i] - df["open"].iloc[i])
        avg_body = abs(df["close"].iloc[max(0,i-10):i].sub(df["open"].iloc[max(0,i-10):i])).abs().mean()
        if body > 1.2 * (avg_body if not pd.isna(avg_body) and avg_body!=0 else body):
            j = i-1
            if df["close"].iloc[j] < df["open"].iloc[j]:
                top = max(df["open"].iloc[j], df["close"].iloc[j])
                bot = min(df["open"].iloc[j], df["close"].iloc[j])
                out.iloc[j, out.columns.get_loc("OB")] = 1
                out.iloc[j, out.columns.get_loc("Top")] = top
                out.iloc[j, out.columns.get_loc("Bottom")] = bot
            if df["close"].iloc[j] > df["open"].iloc[j]:
                top = max(df["open"].iloc[j], df["close"].iloc[j])
                bot = min(df["open"].iloc[j], df["close"].iloc[j])
                out.iloc[j, out.columns.get_loc("OB")] = -1
                out.iloc[j, out.columns.get_loc("Top")] = top
                out.iloc[j, out.columns.get_loc("Bottom")] = bot
    return out


# =========================
# Library-backed feature builder (with fallback)
# =========================
def build_features(df: pd.DataFrame):
    """
    Returns: swings_df, structure_df, fvg_df, ob_df, atr
    structure_df columns normalized to:
        bos_up, bos_down, choch_up, choch_down, trend (bull/bear/neutral), level, broken_index
    """
    atr = compute_atr(df, ATR_WINDOW)

    if USE_SMC_LIB:
        # swings
        swings = smc.swing_highs_lows(df, swing_length=SWING_LEN_LIB)
        # BOS/CHoCH
        st = smc.bos_choch(df, swings, close_break=True)
        # FVG and OB
        fvg_df = smc.fvg(df, join_consecutive=True)
        ob_df  = smc.ob(df, swings, close_mitigation=False)
    else:
        swings = fb_find_swings(df, PIVOT_LEFT_FB, PIVOT_RIGHT_FB)
        st = fb_bos_choch(df, swings, close_break=True)
        fvg_df = fb_fvg(df)
        ob_df  = fb_ob(df, swings)

    # Normalize structure frame
    structure = pd.DataFrame(index=df.index)
    structure["bos_up"]    = (st["BOS"] == 1).astype(bool)
    structure["bos_down"]  = (st["BOS"] == -1).astype(bool)
    structure["choch_up"]  = (st["CHOCH"] == 1).astype(bool)
    structure["choch_down"]= (st["CHOCH"] == -1).astype(bool)
    structure["level"]     = st.get("Level", np.nan)
    structure["broken_index"] = st.get("BrokenIndex", np.nan)

    # derive trend (sticky)
    trend = []
    last = "neutral"
    for i in range(len(structure)):
        if structure["bos_up"].iloc[i]:
            last = "bull"
        elif structure["bos_down"].iloc[i]:
            last = "bear"
        trend.append(last)
    structure["trend"] = trend

    # Clean up NaNs in helper frames (library can return NaNs on non-events)
    for df_ in [swings, fvg_df, ob_df]:
        for c in df_.columns:
            if df_[c].dtype.kind in "fcu":  # float-like
                df_[c] = pd.to_numeric(df_[c], errors="coerce")

    return swings, structure, fvg_df, ob_df, atr


# =========================
# Zones from library outputs (or fallback)
# =========================
def collect_zones(df: pd.DataFrame, fvg_df: pd.DataFrame, ob_df: pd.DataFrame) -> List[Zone]:
    zones: List[Zone] = []
    idx = df.index

    # FVG zones
    for i, t in enumerate(idx):
        f = fvg_df["FVG"].iloc[i] if "FVG" in fvg_df.columns else 0
        if f == 1 or f == -1:
            top = float(fvg_df["Top"].iloc[i])
            bot = float(fvg_df["Bottom"].iloc[i])
            if not (np.isnan(top) or np.isnan(bot)):
                lower, upper = (min(bot, top), max(bot, top))
                kind = "FVG_BULL" if f == 1 else "FVG_BEAR"
                valid_until = idx[min(i + VALID_ZONE_BARS, len(idx)-1)]
                zones.append(Zone(kind, t, lower, upper, valid_until))

    # OB zones
    for i, t in enumerate(idx):
        obv = ob_df["OB"].iloc[i] if "OB" in ob_df.columns else 0
        if obv == 1 or obv == -1:
            top = float(ob_df["Top"].iloc[i])
            bot = float(ob_df["Bottom"].iloc[i])
            if not (np.isnan(top) or np.isnan(bot)):
                lower, upper = (min(bot, top), max(bot, top))
                kind = "OB_BULL" if obv == 1 else "OB_BEAR"
                valid_until = idx[min(i + VALID_ZONE_BARS, len(idx)-1)]
                zones.append(Zone(kind, t, lower, upper, valid_until))

    return zones


# =========================
# Signal Engine
# =========================
def recent_bos_index(structure: pd.DataFrame, i: int, lookback: int) -> Optional[int]:
    lo = max(0, i - lookback)
    seg = structure.iloc[lo:i+1]
    idxs = seg.index[(seg["bos_up"] | seg["bos_down"]).values]
    if len(idxs) == 0:
        return None
    # return absolute integer index of the last BOS
    return structure.index.get_loc(idxs[-1])

def structure_flip_against(structure: pd.DataFrame, i: int, side: str) -> bool:
    if side == "long":
        return bool(structure["bos_down"].iloc[i] or (structure["choch_down"].iloc[i] and structure["trend"].iloc[i] == "bear"))
    else:
        return bool(structure["bos_up"].iloc[i] or (structure["choch_up"].iloc[i] and structure["trend"].iloc[i] == "bull"))

def generate_signals_for_ticker(df: pd.DataFrame, ticker: str) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame(columns=["timestamp","ticker","signal","price","stop","take_profit","r_multiple","reason","trend"])

    swings, structure, fvg_df, ob_df, atr = build_features(df)
    zones = collect_zones(df, fvg_df, ob_df)

    signals = []
    open_positions: List[Position] = []
    idx = df.index

    def active_zones(t: pd.Timestamp) -> List[Zone]:
        return [z for z in zones if t <= z.valid_until]

    for i, t in enumerate(idx):
        o, h, l, c = df["open"].iloc[i], df["high"].iloc[i], df["low"].iloc[i], df["close"].iloc[i]
        trend = structure["trend"].iloc[i]

        # --- Exits first
        for pos in list(open_positions):
            ev = pos.check_exit(t, o, h, l, c, structure_flip_against(structure, i, pos.side))
            if ev is not None:
                signal, when, price, r_mult = ev
                signals.append({
                    "timestamp": when, "ticker": ticker, "signal": signal,
                    "price": round(price, 4), "stop": round(pos.stop_loss, 4),
                    "take_profit": round(pos.take_profit, 4),
                    "r_multiple": round(float(r_mult), 3) if not pd.isna(r_mult) else np.nan,
                    "reason": pos.reason, "trend": trend
                })
        open_positions = [p for p in open_positions if p.open]

        # --- Entry rules (limit to 1 concurrent pos per ticker)
        if len(open_positions) >= MAX_OPEN_TRADES_PER_TICKER:
            continue

        bos_i = recent_bos_index(structure, i, BOS_LOOKBACK)
        if bos_i is None:
            continue

        # 1) RETEST entries (preferred)
        touched_long_zone = any(z.kind.startswith("FVG_BULL") or z.kind.startswith("OB_BULL")
                                for z in active_zones(t) if z.overlaps(l, h))
        touched_short_zone = any(z.kind.startswith("FVG_BEAR") or z.kind.startswith("OB_BEAR")
                                 for z in active_zones(t) if z.overlaps(l, h))

        # 2) FOLLOW-THROUGH option: if directly after a BOS/CHoCH and momentum candle prints
        follow_long = (ENTRY_MODE == "retest_or_follow") and (i - bos_i <= 3) and (structure["bos_up"].iloc[bos_i] or structure["choch_up"].iloc[bos_i]) and (c > o)
        follow_short = (ENTRY_MODE == "retest_or_follow") and (i - bos_i <= 3) and (structure["bos_down"].iloc[bos_i] or structure["choch_down"].iloc[bos_i]) and (c < o)

        # LONG
        if (structure["bos_up"].iloc[bos_i] or structure["choch_up"].iloc[bos_i] or trend == "bull") and (touched_long_zone or follow_long):
            # SL below last swing low or nearest bull zone lower, with ATR cushion
            sw_low = swings["Level"].iloc[i] if "HighLow" in swings.columns and swings["HighLow"].iloc[i] == -1 else np.nan
            sl_cand = []
            if not pd.isna(sw_low): sl_cand.append(sw_low)
            for z in active_zones(t):
                if z.kind.startswith("FVG_BULL") or z.kind.startswith("OB_BULL"):
                    sl_cand.append(z.lower)
            sl_base = min(sl_cand) if len(sl_cand) else (c - 2*atr.iloc[i])
            sl = float(sl_base - ATR_MULT_SL * atr.iloc[i])
            if sl < c:
                tp = float(c + R_MULT_TP * (c - sl))
                pos = Position("long", t, c, sl, tp, "retest_or_follow_long", True, risk_per_share=(c - sl))
                open_positions.append(pos)
                signals.append({
                    "timestamp": t, "ticker": ticker, "signal": "LONG_ENTRY",
                    "price": round(c,4), "stop": round(sl,4), "take_profit": round(tp,4),
                    "r_multiple": round((tp - c) / (c - sl), 2) if (c - sl) > 0 else np.nan,
                    "reason": pos.reason, "trend": trend
                })

        # SHORT
        if (structure["bos_down"].iloc[bos_i] or structure["choch_down"].iloc[bos_i] or trend == "bear") and (touched_short_zone or follow_short):
            sw_high = swings["Level"].iloc[i] if "HighLow" in swings.columns and swings["HighLow"].iloc[i] == 1 else np.nan
            sl_cand = []
            if not pd.isna(sw_high): sl_cand.append(sw_high)
            for z in active_zones(t):
                if z.kind.startswith("FVG_BEAR") or z.kind.startswith("OB_BEAR"):
                    sl_cand.append(z.upper)
            sl_base = max(sl_cand) if len(sl_cand) else (c + 2*atr.iloc[i])
            sl = float(sl_base + ATR_MULT_SL * atr.iloc[i])
            if sl > c:
                tp = float(c - R_MULT_TP * (sl - c))
                pos = Position("short", t, c, sl, tp, "retest_or_follow_short", True, risk_per_share=(sl - c))
                open_positions.append(pos)
                signals.append({
                    "timestamp": t, "ticker": ticker, "signal": "SHORT_ENTRY",
                    "price": round(c,4), "stop": round(sl,4), "take_profit": round(tp,4),
                    "r_multiple": round((c - tp) / (sl - c), 2) if (sl - c) > 0 else np.nan,
                    "reason": pos.reason, "trend": trend
                })

    sig_df = pd.DataFrame(signals)
    if not sig_df.empty:
        sig_df.sort_values(["ticker","timestamp"], inplace=True)
    return sig_df, (swings, structure)


# =========================
# Plotting
# =========================
def plot_smc(df: pd.DataFrame, swings: pd.DataFrame, structure: pd.DataFrame, signals: pd.DataFrame,
             ticker: str, out_dir: str = PLOT_DIR, tail: int = PLOT_MAX_BARS):
    if df.empty:
        return
    os.makedirs(out_dir, exist_ok=True)

    tail_slice = slice(-tail, None) if len(df) > tail else slice(None)
    px = df.iloc[tail_slice].copy()
    st = structure.iloc[tail_slice]
    sw = swings.iloc[tail_slice]
    sig = signals[signals["ticker"] == ticker].copy()
    if not sig.empty:
        sig = sig.set_index("timestamp").sort_index().loc[px.index.min(): px.index.max()]

    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(px.index, px["close"], label="Close", linewidth=1.25)

    # Swings
    if "HighLow" in sw.columns:
        sh = sw[sw["HighLow"]==1].index
        sl = sw[sw["HighLow"]==-1].index
        ax.scatter(sh, df.loc[sh,"high"], marker="^", s=40, label="Swing High", alpha=0.7)
        ax.scatter(sl, df.loc[sl,"low"],  marker="v", s=40, label="Swing Low", alpha=0.7)

    # BOS/CHoCH verticals
    bos_up_idx = st[st["bos_up"]].index
    bos_dn_idx = st[st["bos_down"]].index
    choch_up   = st[st["choch_up"]].index
    choch_dn   = st[st["choch_down"]].index
    for j, t in enumerate(bos_up_idx):
        ax.axvline(t, linestyle="--", alpha=0.25, label="BOS Up" if j==0 else None)
    for j, t in enumerate(bos_dn_idx):
        ax.axvline(t, linestyle="--", alpha=0.25, color="red", label="BOS Down" if j==0 else None)
    for j, t in enumerate(choch_up):
        ax.axvline(t, linestyle=":", alpha=0.2, color="green", label="CHoCH Up" if j==0 else None)
    for j, t in enumerate(choch_dn):
        ax.axvline(t, linestyle=":", alpha=0.2, color="brown", label="CHoCH Down" if j==0 else None)

    # Mark entries/exits
    if not sig.empty:
        le = sig[sig["signal"]=="LONG_ENTRY"]
        lx = sig[sig["signal"]=="LONG_EXIT"]
        se = sig[sig["signal"]=="SHORT_ENTRY"]
        sx = sig[sig["signal"]=="SHORT_EXIT"]
        ax.scatter(le.index, le["price"], marker="^", s=90, label="Long Entry")
        ax.scatter(lx.index, lx["price"], marker="x", s=70, label="Long Exit")
        ax.scatter(se.index, se["price"], marker="v", s=90, label="Short Entry")
        ax.scatter(sx.index, sx["price"], marker="x", s=70, label="Short Exit")

    ax.set_title(f"{ticker} — SMC (BOS/CHoCH & Signals)")
    ax.set_ylabel("Price")
    ax.legend(loc="best", fontsize=9)
    ax.grid(alpha=0.3)
    fig.tight_layout()
    outfile = os.path.join(out_dir, f"{ticker.replace('.','_')}_smc_signals.png")
    fig.savefig(outfile, dpi=140)
    plt.close(fig)


# =========================
# Main
# =========================
def run():
    all_signals = []

    for ticker in TICKERS:
        log.info("Processing %s ...", ticker)
        df = fetch_history(ticker, START_DATE, END_DATE, INTERVAL)
        if df.empty or len(df) < 100:
            log.warning("Skipping %s (insufficient data).", ticker)
            continue

        sig_df, (sw, st) = generate_signals_for_ticker(df, ticker)
        if not sig_df.empty:
            all_signals.append(sig_df)

        # Plot tail
        try:
            plot_smc(df, sw, st, sig_df, ticker, out_dir=PLOT_DIR, tail=PLOT_MAX_BARS)
        except Exception as e:
            log.error("Plot failed for %s: %s", ticker, e)

    if all_signals:
        out = pd.concat(all_signals, ignore_index=True)
        out.sort_values(["ticker","timestamp"], inplace=True)
        out.to_csv(OUT_CSV, index=False)
        log.info("Saved signals to %s (%d rows).", OUT_CSV, len(out))
    else:
        log.info("No signals generated. Try: INTERVAL='1h' or '30m', ENTRY_MODE='retest_or_follow', SWING_LEN_LIB=8, or extend START_DATE.")

if __name__ == "__main__":
    run()


2025-09-05 15:11:42,920 | INFO | smartmoneyconcepts detected. Using library indicators where possible.
2025-09-05 15:11:42,924 | INFO | Processing 360ONE.NS ...
2025-09-05 15:11:43,677 | ERROR | Plot failed for 360ONE.NS: 'ticker'
2025-09-05 15:11:43,678 | INFO | Processing 3MINDIA.NS ...
2025-09-05 15:11:44,357 | ERROR | Plot failed for 3MINDIA.NS: 'ticker'
2025-09-05 15:11:44,358 | INFO | Processing ABB.NS ...
2025-09-05 15:11:45,352 | ERROR | Plot failed for ABB.NS: 'ticker'
2025-09-05 15:11:45,352 | INFO | Processing ACC.NS ...
2025-09-05 15:11:46,112 | ERROR | Plot failed for ACC.NS: 'ticker'
2025-09-05 15:11:46,112 | INFO | Processing ACMESOLAR.NS ...
2025-09-05 15:11:46,358 | ERROR | Plot failed for ACMESOLAR.NS: 'ticker'
2025-09-05 15:11:46,358 | INFO | Processing AIAENG.NS ...
2025-09-05 15:11:47,066 | ERROR | Plot failed for AIAENG.NS: 'ticker'
2025-09-05 15:11:47,067 | INFO | Processing APLAPOLLO.NS ...
2025-09-05 15:11:47,878 | ERROR | Plot failed for APLAPOLLO.NS: 'ticker'